In [94]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [95]:
mx.context.num_gpus()

0

In [96]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/TFT_energy.csv')

In [97]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [98]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'time'], infer_datetime_format=True)

In [99]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0,2015-01-01 00:00:00+00:00
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0,2015-01-01 01:00:00+00:00
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0,2015-01-01 02:00:00+00:00
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0,2015-01-01 03:00:00+00:00
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0,2015-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [100]:
df.columns

Index(['time', 'price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump',
       'timestamp'],
      dtype='object')

In [101]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [102]:
train_df.shape

(28050, 28)

In [103]:
test_df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
28050,2018-03-14 18:00:00+00:00,50.95,37.61,1068.0,5319.0,35044.0,0.0,345.0,102.0,1006.0,...,40.0,1849.0,220.0,1.0,1.0,8.0,14265.0,6.0,761.0,2018-03-14 18:00:00+00:00
28051,2018-03-14 19:00:00+00:00,59.73,48.21,1126.0,5292.0,36230.0,0.0,351.0,101.0,1005.0,...,40.0,1821.0,220.0,4.0,2.0,8.0,13306.0,5.0,149.0,2018-03-14 19:00:00+00:00
28052,2018-03-14 20:00:00+00:00,50.89,41.11,1054.0,4318.0,35065.0,0.0,351.0,103.0,1005.0,...,40.0,1853.0,220.0,1.0,2.0,5.0,12970.0,7.0,168.0,2018-03-14 20:00:00+00:00
28053,2018-03-14 21:00:00+00:00,45.94,36.72,963.0,4047.0,32328.0,0.0,350.0,104.0,1004.0,...,20.0,1853.0,210.0,1.0,1.0,1.0,13090.0,3.0,811.0,2018-03-14 21:00:00+00:00
28054,2018-03-14 22:00:00+00:00,30.59,21.20,853.0,3726.0,29528.0,0.0,329.0,95.0,1004.0,...,0.0,1826.0,220.0,0.0,8.0,5.0,13408.0,6.0,2007.0,2018-03-14 22:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [104]:
train_ds = PandasDataset(
    train_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [105]:
test_ds = PandasDataset(
    test_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### training

In [108]:
K = 6
n_hours = 24*3

In [ ]:
trainer=Trainer(
    learning_rate=1e-2,
    epochs=50,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    num_layers=1, # default: 2
    num_cells=128,  # default: 40
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=64,
    trainer=trainer,
    dropout_rate=0.4
)

model = estimator.train(train_ds)

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████       | 95/100 [04:20<00:13,  2.69s/it, epoch=50/50, avg_epoch_loss=3.52]

#### predicted y

In [83]:
forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
forecasts = list(forecast_it)
tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [84]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [85]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832]
(6,)
------
1
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122]
(12,)
------
2
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375     ]
(18,)
------
3


/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375
 68.38999939 67.73999786 61.51499939 50.29499817 47.375      42.11500168]
(24,)
------
4
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375
 68.38999939 67.73999786 61.51499939 50.29499817 47.375      42.11500168
 66.29000092 59.84000015 50.96500015 48.80500031 43.43500137 43.29999924]
(30,)
------
5
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375
 68.38999939 67.73999786 61.51499939 50.29499817 47.375      42.11500168
 66.29000092 59.84000015 50.965000

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375
 68.38999939 67.73999786 61.51499939 50.29499817 47.375      42.11500168
 66.29000092 59.84000015 50.96500015 48.80500031 43.43500137 43.29999924
 58.84500122 50.41500092 50.54999924 44.88499832 43.88000107 45.13499832
 51.00500107 50.32500076 45.04999924 44.22000122 44.99000168 46.63999939
 49.25500107 43.72999954 42.88499832 44.77999878 46.51499939 47.24499893]
(48,)
------
8
[59.06000137 61.72499847 62.54000092 61.49499893 60.22000122 54.38499832
 65.06999969 65.41500092 63.52000046 62.11500168 56.39500046 46.22000122
 69.15499878 66.15499878 65.47499847 58.43500137 48.68000031 46.375
 68.38999939 67.73999786 61.51499939 50.29499817 47.375      42.11500168
 66.29000092 59.84000015 50.96500015 48.80500031 43.43500137 43.29999924
 58.84500122 50.41500092 50.54999924 44.88499

...

/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[59.06000137 61.72499847 62.54000092 ... 85.44999695 85.23999786
 76.45500183]
(20448,)
------
3408
[59.06000137 61.72499847 62.54000092 ... 85.80999756 77.88999939
 73.33499908]
(20454,)
------
3409
[59.06000137 61.72499847 62.54000092 ... 76.47000122 74.43000031
 73.08499908]
(20460,)
------
3410
[59.06000137 61.72499847 62.54000092 ... 73.375      74.73000336
 74.36000061]
(20466,)
------
3411


/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


[59.06000137 61.72499847 62.54000092 ... 74.77500153 76.23000336
 76.25      ]
(20472,)
------
3412
[59.06000137 61.72499847 62.54000092 ... 74.55500031 75.05999756
 75.86000061]
(20478,)
------
3413
[59.06000137 61.72499847 62.54000092 ... 72.94999695 74.16500092
 65.11000061]
(20484,)
------
3414
[59.06000137 61.72499847 62.54000092 ... 70.43000031 62.70000076
 54.35499954]
(20490,)
------
3415


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [86]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(6936, 6)

In [87]:
test_ds_predict_q50_supervised[0:5]

array([[59.06000137, 61.72499847, 62.54000092, 61.49499893, 60.22000122,
        54.38499832],
       [65.06999969, 65.41500092, 63.52000046, 62.11500168, 56.39500046,
        46.22000122],
       [69.15499878, 66.15499878, 65.47499847, 58.43500137, 48.68000031,
        46.375     ],
       [68.38999939, 67.73999786, 61.51499939, 50.29499817, 47.375     ,
        42.11500168],
       [66.29000092, 59.84000015, 50.96500015, 48.80500031, 43.43500137,
        43.29999924]])

#### actual y

In [88]:
from helper import series_to_supervised, stage_series_to_supervised

In [89]:
test_df_supervised = series_to_supervised(test_df[['price']], n_hours, K)

In [90]:
test_df_supervised.iloc[:, -K:].shape

(6936, 6)

In [91]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [92]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5)
28122,63.57,73.69,67.93,64.33,59.28,55.41
28123,73.69,67.93,64.33,59.28,55.41,53.16
28124,67.93,64.33,59.28,55.41,53.16,49.20
28125,64.33,59.28,55.41,53.16,49.20,48.17
28126,59.28,55.41,53.16,49.20,48.17,46.76
...,...,...,...,...,...,...
35053,71.95,70.85,71.36,75.13,77.61,77.02
35054,70.85,71.36,75.13,77.61,77.02,76.16
35055,71.36,75.13,77.61,77.02,76.16,74.30
35056,75.13,77.61,77.02,76.16,74.30,69.89


#### performance on test set

In [93]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 5.353
RMSE = 7.178
